In [133]:
import requests, praw, re, time
from bs4 import BeautifulSoup 
from datetime import datetime
import pandas as pd 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import nltk
from nltk.corpus import stopwords 
from config import indeed_query,r_client_id,r_secret_key,r_dev,r_username,r_pw

In [134]:
reddit = praw.Reddit(
    client_id=r_client_id,
    client_secret=r_secret_key,
    password=r_pw,
    user_agent=f'testscript by u/{r_dev}',
    username=r_dev,
)
print(reddit.user.me())

Jazzy_Danger_1230


In [185]:
def getsymb(s):
    lst = []
    s = re.sub('[^A-Z\s]','', s)   
    for w in s.split(' '):  
        if re.match('\w{2,4}',w): 
            lst.append(w)
    return ','.join(lst)
getsymb('1000$ of BB will take it to 300 billion market. TO THE MOON to see SPY in the 400-405 range Bb & NOK ready for takeoff')

'BB,TO,THE,MOON,SPY,NOK'

In [186]:
submissions=reddit.subreddit("wallstreetbets").search("Daily Discussion Thread", limit=4, sort='new')

In [187]:
lst = []
for submission in submissions:
    # print(submission.title)    
    # submission.comments.replace_more(limit=None)
    submission.comment_sort = 'new'
    for comment in submission.comments: 
        try:
            lst.append(
                {
                    'submission_title':submission.title
                    ,'comment_body':comment.body
                    ,'time':datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                    ,'author': comment.author
                    ,'score':comment.score
                    ,'permalink':comment.permalink
                    ,'replies':comment.replies
                    ,'id':comment.id 
                }
            ) 
        except:
            # print('err' + comment.id  )
            pass    
df = pd.DataFrame(lst)
df['SYMB'] = df.comment_body.apply(getsymb)  
df['time'] = pd.to_datetime(df.time)
df['day'] = df.time.apply(lambda s: s.strftime('%d'))
df['hour'] = df.time.apply(lambda s: s.strftime('%I')) 
df.permalink = 'https://www.reddit.com/' + df.permalink
df.sort_values('time', ascending=False)

,submission_title,comment_body,time,author,score,permalink,replies,id,SYMB,day,hour
0,"Daily Discussion Thread for June 04, 2021","If you had a bad week, don’t worry. Next week ...",2021-06-04 22:44:43,LordOfBirds,2,https://www.reddit.com//r/wallstreetbets/comme...,(),h0mg9zz,,04,10
1,"Daily Discussion Thread for June 04, 2021",What happened to your diamond hands😕😒,2021-06-04 22:41:54,Heavy-TapBabyCakez,2,https://www.reddit.com//r/wallstreetbets/comme...,(),h0mfxpz,,04,10
2,"Daily Discussion Thread for June 04, 2021",BB following the AMC pattern. Feel it will bou...,2021-06-04 22:37:45,Kranthikari007,12,https://www.reddit.com//r/wallstreetbets/comme...,(h0mfwhn),h0mffe4,"BB,AMC",04,10
3,"Daily Discussion Thread for June 04, 2021",Buy Monday sell on Friday. \nRinse & Repeat.,2021-06-04 22:31:36,Soggy_r0ck,3,https://www.reddit.com//r/wallstreetbets/comme...,(h0mfhx9),h0meofo,,04,10
4,"Daily Discussion Thread for June 04, 2021",You guys should have sold.,2021-06-04 22:26:13,Soggy_r0ck,-5,https://www.reddit.com//r/wallstreetbets/comme...,(),h0me0cn,,04,10
...,...,...,...,...,...,...,...,...,...,...,...
650,"What Are Your Moves Tomorrow, June 02, 2021",BB $14s mothafuckas,2021-06-02 09:58:23,Flashy_Mix_1987,5,https://www.reddit.com//r/wallstreetbets/comme...,(),h0aow84,BB,02,09
651,"What Are Your Moves Tomorrow, June 02, 2021",BB BB BB,2021-06-02 09:58:03,phly,6,https://www.reddit.com//r/wallstreetbets/comme...,(),h0aoveu,"BB,BB,BB",02,09
652,"What Are Your Moves Tomorrow, June 02, 2021",[deleted],2021-06-02 09:57:56,None,17,https://www.reddit.com//r/wallstreetbets/comme...,"(h0apzcp, h0aphl2)",h0aov5f,,02,09
653,"What Are Your Moves Tomorrow, June 02, 2021",$60 by EOD?,2021-06-02 09:57:53,bearrfuk,12,https://www.reddit.com//r/wallstreetbets/comme...,"(h0ap3cg, h0aozcs, <MoreComments count=1, chil...",h0aov05,EOD,02,09


ser = df.author.value_counts()
names = [r for r in list(ser.index.values)] 
df1 = df.loc[ df['author'].isin( names[:5] )  ]  
df1.author = df1.author.apply(lambda x: str(x.name))
df1=df1.groupby('author').describe()

In [162]:
df1=df
df1 = df1.loc[ (df1['time'] > "2021-06-01"  ) & (df1['time'] < "2021-06-05")  ]
df1 = df1.loc[ df1['SYMB'].str.contains('BB,')  ]
for p in df1.permalink:
    print(p)

https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l8cwl/
https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l8cog/
https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l867h/
https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l826a/
https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l7zzu/
https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l7sdg/
https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l7re5/
https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l7rd4/
https://www.reddit.com//r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/h0l7hyx/
https://www.reddit.com//r/wallstreetb

,submission_title,comment_body,time,author,score,permalink,replies,id,SYMB
0,"Daily Discussion Thread for June 04, 2021",I’m a in on CLNE for a thousand Bob.,2021-06-04 13:48:36,TrioDoge,1,/r/wallstreetbets/comments/ns1gd8/daily_discus...,(),h0kf7yj,CLNE
1,"Daily Discussion Thread for June 04, 2021","So, BB? Hmmmph",2021-06-04 13:48:35,Live-Ad6746,1,/r/wallstreetbets/comments/ns1gd8/daily_discus...,(),h0kf7vs,BB
2,"Daily Discussion Thread for June 04, 2021",What the hell did you guys do to $BB?,2021-06-04 13:48:35,NewbieDoobie18,1,/r/wallstreetbets/comments/ns1gd8/daily_discus...,(),h0kf7vi,BB
3,"Daily Discussion Thread for June 04, 2021",Dont sleep on CLOV !!,2021-06-04 13:48:34,DependentHot1829,1,/r/wallstreetbets/comments/ns1gd8/daily_discus...,(),h0kf7u5,CLOV
4,"Daily Discussion Thread for June 04, 2021",My bb is testing my faith. Too bad I’m loyal a...,2021-06-04 13:48:34,Tendie_rizer,1,/r/wallstreetbets/comments/ns1gd8/daily_discus...,(),h0kf7tn,
...,...,...,...,...,...,...,...,...,...
4586,"Daily Discussion Thread for May 18, 2021",It’s 12/31/21. SPY finishes the year down 0.2%.,2021-05-18 19:55:59,Griffinhunters,5,/r/wallstreetbets/comments/nf6oeg/daily_discus...,(),gylyl21,SPY
4587,"Daily Discussion Thread for May 18, 2021",SPY calls at the buzzer??,2021-05-18 19:55:59,EvaUnit343,2,/r/wallstreetbets/comments/nf6oeg/daily_discus...,(gylyp0v),gylyl1v,SPY
4588,"Daily Discussion Thread for May 18, 2021",W H Y,2021-05-18 19:55:58,lizlegit000,3,/r/wallstreetbets/comments/nf6oeg/daily_discus...,(),gylykxb,
4589,"Daily Discussion Thread for May 18, 2021",AMC is dripping with wack juice. Buying puts t...,2021-05-18 19:55:57,Pester_Stone,-1,/r/wallstreetbets/comments/nf6oeg/daily_discus...,(),gylyku6,AMC


,submission_title,comment_body,time,author,score,permalink,replies,id,SYMB,day,hour
0,"Daily Discussion Thread for June 04, 2021",I’m a in on CLNE for a thousand Bob.,2021-06-04 13:48:36,TrioDoge,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7yj,CLNE,04,01
2,"Daily Discussion Thread for June 04, 2021",What the hell did you guys do to $BB?,2021-06-04 13:48:35,NewbieDoobie18,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7vi,BB,04,01
1,"Daily Discussion Thread for June 04, 2021","So, BB? Hmmmph",2021-06-04 13:48:35,Live-Ad6746,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7vs,BB,04,01
3,"Daily Discussion Thread for June 04, 2021",Dont sleep on CLOV !!,2021-06-04 13:48:34,DependentHot1829,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7u5,CLOV,04,01
6,"Daily Discussion Thread for June 04, 2021",Cmon bb kick girl,2021-06-04 13:48:34,OneEyedKing808,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7sb,,04,01
...,...,...,...,...,...,...,...,...,...,...,...
4586,"Daily Discussion Thread for May 18, 2021",It’s 12/31/21. SPY finishes the year down 0.2%.,2021-05-18 19:55:59,Griffinhunters,5,https://www.reddit.com//r/wallstreetbets/comme...,(),gylyl21,SPY,18,07
4587,"Daily Discussion Thread for May 18, 2021",SPY calls at the buzzer??,2021-05-18 19:55:59,EvaUnit343,2,https://www.reddit.com//r/wallstreetbets/comme...,(gylyp0v),gylyl1v,SPY,18,07
4588,"Daily Discussion Thread for May 18, 2021",W H Y,2021-05-18 19:55:58,lizlegit000,3,https://www.reddit.com//r/wallstreetbets/comme...,(),gylykxb,,18,07
4589,"Daily Discussion Thread for May 18, 2021",AMC is dripping with wack juice. Buying puts t...,2021-05-18 19:55:57,Pester_Stone,-1,https://www.reddit.com//r/wallstreetbets/comme...,(),gylyku6,AMC,18,07


https://www.reddit.com//r/wallstreetbets/comments/nqi9f6/daily_discussion_thread_for_june_02_2021/h0f9a6d/
https://www.reddit.com//r/wallstreetbets/comments/nqi9f6/daily_discussion_thread_for_june_02_2021/h0d6su8/
https://www.reddit.com//r/wallstreetbets/comments/npqenw/daily_discussion_thread_for_june_01_2021/h09b3a7/
https://www.reddit.com//r/wallstreetbets/comments/npqenw/daily_discussion_thread_for_june_01_2021/h096l5t/
https://www.reddit.com//r/wallstreetbets/comments/npqenw/daily_discussion_thread_for_june_01_2021/h08ch4b/
https://www.reddit.com//r/wallstreetbets/comments/npbyt0/what_are_your_moves_tomorrow_june_01_2021/h06nrrl/
https://www.reddit.com//r/wallstreetbets/comments/npbyt0/what_are_your_moves_tomorrow_june_01_2021/h06ilnh/
https://www.reddit.com//r/wallstreetbets/comments/npbyt0/what_are_your_moves_tomorrow_june_01_2021/h06dw1z/


In [ ]:
p=0
url = f'https://washingtondc.craigslist.org/d/bicycles/search/bia?s={p}'
soup = BeautifulSoup(requests.get(url).text, "html.parser")

   
    